# Capstone Project - The Battle of Neighborhoods (Week 2)

### Download of all the dependencies that we need

In [4]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import folium # plotting library

from bs4 import BeautifulSoup
from IPython.display import display_html
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Libraries imported.


### Scraping the Wikipedia page for the list of boroughs of Nantes Metropolis

In [6]:
# Download data and parse it:
request = requests.get('https://fr.wikipedia.org/wiki/Nantes_M%C3%A9tropole')
soup = BeautifulSoup(request.text, 'html.parser')
# First remove Falsey values
table_soup = soup.find_all('table')
filtered_table_soup = [table for table in table_soup if table.caption is not None]

required_table = None

for table in filtered_table_soup:
    if str(table.caption.string).strip() == 'Liste des communes de l’intercommunalité':
        required_table = table
        break
# Then display the table
table_str =  str(table)
display_html(table_str, raw=True)

Nom,Code Insee,Gentilé,Superficie (km2),Population (dernière pop. légale),Densité (hab./km2)
Nantes(siège),44109,Nantais,"65,19",‍314 138 (2018),4 819
Basse-Goulaine,44009,Goulainais,"13,74",‍9 093 (2018),662
Bouaye,44018,Boscéens,"13,83",‍7 914 (2018),572
Bouguenais,44020,Bouguenaisiens,"31,5",‍19 658 (2018),624
Brains,44024,Brennois,"15,31",‍2 836 (2018),185
Carquefou,44026,Carquefoliens,"43,42",‍20 032 (2018),461
La Chapelle-sur-Erdre,44035,Chapelains,"33,42",‍19 526 (2018),584
Couëron,44047,Coueronnais,"44,03",‍21 862 (2018),497
Indre,44074,Indrais,"4,72",‍4 037 (2018),855
Mauves-sur-Loire,44094,Malviens,"14,75",‍3 230 (2018),219


## Data Preparation

In [7]:
# The HTML table is converted to a Pandas DataFrame
df_init = pd.read_html(table_str, flavor= 'html5lib')
df = df_init[0]
df.head()

,Nom,Code Insee,Gentilé,Superficie (km2),Population (dernière pop. légale),Densité (hab./km2)
0,Nantes(siège),44109,Nantais,6519,314 138 (2018),4 819
1,Basse-Goulaine,44009,Goulainais,1374,9 093 (2018),662
2,Bouaye,44018,Boscéens,1383,7 914 (2018),572
3,Bouguenais,44020,Bouguenaisiens,315,19 658 (2018),624
4,Brains,44024,Brennois,1531,2 836 (2018),185


## Data Cleaning

In [ ]:
# We will only keep the boroughs that are adjacent to the center of Nantes : 